# 🚯 Lecture 11 Lab: Logistic regression and spam detection

<img src="https://github.com/joshuagrossman/mse125-labs-public/blob/main/hw5/img/spam-email.png?raw=1" alt= “spam-email” width="500" />

## ✅ Setup and data import
In this lab, we will work with a [classic dataset](https://archive.ics.uci.edu/dataset/94/spambase) of 4,601 emails classified as spam or not spam.

In [9]:
# Load in additional functions
library(tidyverse)
library(lubridate)

# Use three digits past the decimal point,
# and don't use scientific notation.
options(digits = 3, scipen = 999)

# Format plots with a white background and dark features.
theme_set(theme_bw())

# Increase the default text size of plots.
# If you are *not* working in Google Colab, we recommend commenting
# out this line of code.
theme_update(text = element_text(size = 20))

# Increase the default plot width and height.
# If you are *not* working in Google Colab, we recommend commenting
# out this line of code.
options(repr.plot.width=12, repr.plot.height=8)

# Read in the data
spam = read_csv('https://jdgrossman.com/assets/spam.csv')

# peek at 10 random rows
sample_n(spam, 10)


SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (ipython-input-1360378051.py, line 20)

## ♨️ Warm up

How many emails are in the database?

What fraction of the emails in the database are spam?

Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?

In [10]:
# Your code here!

import pandas as pd

# Load the dataset
url = 'https://jdgrossman.com/assets/spam.csv'
spam = pd.read_csv(url, encoding='latin1')

# Total number of emails
n_emails = spam.shape[0]
print(f"Total emails: {n_emails}")

# Fraction of emails that are spam
fraction_spam = spam['is_spam'].mean()
print(f"Fraction of emails that are spam: {fraction_spam:.3f} ({fraction_spam*100:.1f}%)")

# Email(s) with the highest percentage of words matching "money"
max_money = spam['money'].max()
emails_max_money = spam[spam['money'] == max_money]

print(f"\nHighest 'money' percentage: {max_money}%")
print("Email(s) with highest 'money' percentage:")
print(emails_max_money)

Total emails: 4601
Fraction of emails that are spam: 0.394 (39.4%)

Highest 'money' percentage: 12.5%
Email(s) with highest 'money' percentage:
     make  address  all   3d  our  over  remove  internet  order  mail  ...  \
545   0.0      0.0  0.0  0.0  0.0   0.0     0.0       0.0    0.0   0.0  ...   

     char_semicolon  char_left_paren  char_left_bracket  char_exclamation  \
545             0.0              0.0                0.0               0.0   

     char_dollar  char_pound  capital_run_length_average  \
545          0.0         0.0                         1.0   

     capital_run_length_longest  capital_run_length_total  is_spam  
545                           1                         2        1  

[1 rows x 58 columns]


## 🎲 Linear probability models (LPMs)

Fit a linear regression model to the spam data with the `lm` function.

Use the following covariates to predict the likelihood that an email is spam:
- `char_dollar`
- `credit`
- `money`
- `re`

How would you interpret the model coefficients for the intercept and for `char_dollar`?

- Note: `char_dollar` represents the percentage of characters in the email that match `$`.

In [11]:
import statsmodels.api as sm

# Select covariates
X = spam[['char_dollar', 'credit', 'money', 're']]
y = spam['is_spam']  # response variable

# Add an intercept
X = sm.add_constant(X)

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# Show summary
print(model.summary())




                            OLS Regression Results                            
Dep. Variable:                is_spam   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     250.3
Date:                Tue, 18 Nov 2025   Prob (F-statistic):          8.51e-195
Time:                        06:08:52   Log-Likelihood:                -2780.3
No. Observations:                4601   AIC:                             5571.
Df Residuals:                    4596   BIC:                             5603.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.3346      0.007     45.696      

Using your linear probability model and the `predict` function, predict the in-sample probability that each email is spam.

What is the smallest predicted probability? The largest? Do you notice any issues with these predictions?

In [4]:
# Predict in-sample probabilities
pred_probs = model.predict(X)

# Add predictions to the dataframe (optional)
spam['pred_prob'] = pred_probs

# Check min and max predicted probabilities
min_prob = pred_probs.min()
max_prob = pred_probs.max()
print(f"Smallest predicted probability: {min_prob:.3f}")
print(f"Largest predicted probability: {max_prob:.3f}")



Smallest predicted probability: -0.813
Largest predicted probability: 3.849



**Main issue**: LPM can produce invalid probabilities outside the [0,1] range, which makes it unsuitable for precise probability predictions.

## 🎰 Odds functions

Write two functions:
- A function to convert probabilities to odds.
- A function to convert odds to probabilities

Test your functions by making sure that 2:1 odds returns a 2/3 probability, and vice versa.

Finally, suppose my probability of winning is 60%. If I double my odds of winning, what is my new probability of winning?

In [5]:
# Function to convert probability to odds
def prob_to_odds(p):
    """
    Convert probability p (0 <= p <= 1) to odds.
    """
    return p / (1 - p)

# Function to convert odds to probability
def odds_to_prob(o):
    """
    Convert odds o to probability.
    """
    return o / (1 + o)

# ------------------------------
# Test the functions
# 2:1 odds should return 2/3 probability
odds_test = 2
prob_from_odds = odds_to_prob(odds_test)
print(f"Odds {odds_test}:1 → Probability = {prob_from_odds:.3f}")

# 2/3 probability should return 2:1 odds
prob_test = 2/3
odds_from_prob = prob_to_odds(prob_test)
print(f"Probability {prob_test:.3f} → Odds = {odds_from_prob:.2f}:1")

# Suppose probability of winning is 60%
p = 0.6

# Convert to odds
current_odds = prob_to_odds(p)

# Double the odds
doubled_odds = 2 * current_odds

# Convert back to probability
new_prob = odds_to_prob(doubled_odds)
print(f"\nOriginal probability: {p:.2f}")
print(f"Doubled odds probability: {new_prob:.3f}")



Odds 2:1 → Probability = 0.667
Probability 0.667 → Odds = 2.00:1

Original probability: 0.60
Doubled odds probability: 0.750



If I double my odds of winning from 60% probability, my new probability of winning = 75%.

## 🪙 Fitting a logistic regression model

We can fit a logistic regression model with the same covariates as above with the following code:

In [6]:
import pandas as pd
import statsmodels.api as sm

# Load the dataset
url = 'https://jdgrossman.com/assets/spam.csv'
spam = pd.read_csv(url, encoding='latin1')

# Select covariates and response
X = spam[['char_dollar', 'credit', 'money', 're']]
y = spam['is_spam']

# Add an intercept
X = sm.add_constant(X)

# Fit logistic regression model
logit_model = sm.Logit(y, X).fit()

# Show model summary
print(logit_model.summary())

# Predict in-sample probabilities
spam['pred_prob_logit'] = logit_model.predict(X)

# Display min and max predicted probabilities
print(f"\nSmallest predicted probability: {spam['pred_prob_logit'].min():.3f}")
print(f"Largest predicted probability: {spam['pred_prob_logit'].max():.3f}")

Optimization terminated successfully.
         Current function value: 0.481178
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                is_spam   No. Observations:                 4601
Model:                          Logit   Df Residuals:                     4596
Method:                           MLE   Df Model:                            4
Date:                Tue, 18 Nov 2025   Pseudo R-squ.:                  0.2824
Time:                        06:06:48   Log-Likelihood:                -2213.9
converged:                       True   LL-Null:                       -3085.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -1.0666      0.043    -24.680      0.000      -1.151      -0.982
char_dollar    11.8176    

Interpret the intercept and `money` coefficients for the logistic regression model three different ways:
1. On the log odds scale
2. On the odds scale (by exponentiating the coefficients)
3. On the probability scale (using either the odds functions you wrote, or the divide by 4 trick).

Tip: Use the `coef` function to extract coefficients from the model.

In [7]:
import numpy as np

# Extract coefficients
coef = logit_model.params
intercept_coef = coef['const']
money_coef = coef['money']

# Log-odds scale
print(f"Log-odds scale:")
print(f"Intercept (const): {intercept_coef:.3f}")
print(f"Money: {money_coef:.3f}")

# Odds scale (exponentiate coefficients)
intercept_odds = np.exp(intercept_coef)
money_odds = np.exp(money_coef)
print(f"\nOdds scale:")
print(f"Intercept odds: {intercept_odds:.3f}")
print(f"Money odds ratio: {money_odds:.3f}")

# Probability scale
def odds_to_prob(o):
    return o / (1 + o)

intercept_prob = odds_to_prob(intercept_odds)
# For money, approximate probability effect (depends on other predictors)
money_prob = odds_to_prob(money_odds)

print(f"\nProbability scale (approx):")
print(f"Intercept probability: {intercept_prob:.3f}")
print(f"Money coefficient effect (approx probability if others=0): {money_prob:.3f}")




Log-odds scale:
Intercept (const): -1.067
Money: 1.993

Odds scale:
Intercept odds: 0.344
Money odds ratio: 7.340

Probability scale (approx):
Intercept probability: 0.256
Money coefficient effect (approx probability if others=0): 0.880
